# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
from statistics import mean

from secretsAPI import IEX_CLOUD_API_TOKEN

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 74169625,
 'calculationPrice': 'close',
 'change': -1,
 'changePercent': -0.00625,
 'close': 166,
 'closeSource': 'ciolaiff',
 'closeTime': 1728952440513,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 161.75,
 'delayedPriceTime': 1702162621264,
 'extendedChange': -0.14,
 'extendedChangePercent': -0.00087,
 'extendedPrice': 162.64,
 'extendedPriceTime': 1719805297288,
 'high': 169.01,
 'highSource': 'm5eeuleirnpyted d1cia  ',
 'highTime': 1676924550015,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 163.06,
 'iexCloseTime': 1697871629366,
 'iexLastUpdated': 1671896706403,
 'iexMarketPercent': 0.02113276940474554,
 'iexOpen': 162.91,
 'iexOpenTime': 1735821576678,
 'iexRealtimePrice': 168.46,
 'iexRealtimeSize': 220,
 'iexVolume': 1430002,
 'lastTradeTime': 1701490402221,
 'latestPrice': 165.87,
 'latestSource': 'Close',
 'latestTime': 'August 1, 2022',
 'latestUpdate': 1709216604038,
 'latestVolume': 70989179

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

26.85

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batchCall = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchCall).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                            pd.Series(
                            [
                                symbol,
                                data[symbol]['quote']['latestPrice'],
                                data[symbol]['quote']['peRatio'],
                                'N/A'
                            ], index = my_columns), ignore_index=True
                        )
final_dataframe

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\L

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\L

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\L

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\L

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\L

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3971042664.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AAPL,162.08,26.43,N/A
1,MSFT,284.07,28.85,N/A
2,AMZN,141.28,3.26,N/A
3,TSLA,898.30,110.74,N/A
4,BRK.B,298.29,None,N/A
...,...,...,...,...
498,PENN,35.19,16.44,N/A
499,VNO,30.70,48.5,N/A
500,ALK,47.29,12.67,N/A
501,PVH,63.73,4.67,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.28,2.35,N/A
1,EBAY,49.10,2.79,N/A
2,AMZN,141.28,3.26,N/A
3,PARA,24.18,4.08,N/A
4,COF,113.64,4.45,N/A
5,AIG,52.35,4.66,N/A
6,PVH,63.73,4.67,N/A
7,MRNA,168.42,4.76,N/A
8,DISH,17.28,4.97,N/A
9,LYB,87.41,5.01,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def getPortfolioSize():
    invalid_portfolio_size = True
    while (invalid_portfolio_size):
        portfolio_size = input("Enter Portfolio Value: ")
        try:
            val = float(portfolio_size)
            invalid_portfolio_size = False
        except ValueError:
            print("Input must be a valid number")
    return val

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_size = getPortfolioSize()

Enter Portfolio Value: 2500000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.28,2.35,1306
1,EBAY,49.10,2.79,1018
2,AMZN,141.28,3.26,353
3,PARA,24.18,4.08,2067
4,COF,113.64,4.45,439
5,AIG,52.35,4.66,955
6,PVH,63.73,4.67,784
7,MRNA,168.42,4.76,296
8,DISH,17.28,4.97,2893
9,LYB,87.41,5.01,572


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batchCall =  f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batchCall).json()
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Deprecation, and Amoritization
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue'] / 1e12
ebitda = data[symbol]['advanced-stats']['EBITDA'] / 1e12
ev_to_ebitda = enterprise_value / ebitda

# Enterprise Value divided by Gross Profit
gross_profit = data[symbol]['advanced-stats']['grossProfit'] / 1e12
ev_to_gross_profit = enterprise_value / gross_profit
print(ev_to_gross_profit)

16.31699856295766


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares To Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batchCall = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchCall).json()
    
    for symbol in symbol_string.split(','):
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_columns),
                ignore_index = True
        )

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_22260\3713901119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(


In [13]:
rv_dataframe

,Ticker,Price,Number of Shares To Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,165.39,N/A,27.23,N/A,39.43,N/A,7.11,N/A,20.645155,N/A,15.693638,N/A,N/A
1,MSFT,283.58,N/A,29.22,N/A,12.75,N/A,10.74,N/A,21.015992,N/A,15.052025,N/A,N/A
2,AMZN,135.75,N/A,3.28,N/A,10.75,N/A,2.88,N/A,25.330172,N/A,6.740078,N/A,N/A
3,TSLA,900.61,N/A,108.72,N/A,26.1,N/A,14.66,N/A,70.097480,N/A,54.744528,N/A,N/A
4,BRK.B,296.30,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,PENN,34.65,N/A,16.6,N/A,1.48,N/A,0.9554,N/A,6.875277,N/A,3.694826,N/A,N/A
499,VNO,30.60,N/A,49.31,N/A,1.2,N/A,3.56,N/A,15.036700,N/A,7.911411,N/A,N/A
500,ALK,45.81,N/A,12.72,N/A,1.6,N/A,0.7981,N/A,9.247482,N/A,0.737681,N/A,N/A
501,PVH,63.44,N/A,4.54,N/A,0.788,N/A,0.4602,N/A,4.758376,N/A,1.103400,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)])

8

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe.fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares To Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric]) / 100

In [18]:
rv_dataframe

,Ticker,Price,Number of Shares To Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,165.39,N/A,27.230000,0.634195,39.430000,0.970179,7.110000,0.823062,20.645155,0.725646,15.693638,0.880716,N/A
1,MSFT,283.58,N/A,29.220000,0.675944,12.750000,0.87674,10.740000,0.904573,21.015992,0.729622,15.052025,0.868787,N/A
2,AMZN,135.75,N/A,3.280000,0.039761,10.750000,0.836978,2.880000,0.484095,25.330172,0.840954,6.740078,0.469185,N/A
3,TSLA,900.61,N/A,108.720000,0.980119,26.100000,0.940358,14.660000,0.946322,70.097480,0.984095,54.744528,0.998012,N/A
4,BRK.B,296.30,N/A,26.826081,0.615308,26.826081,0.949304,26.826081,0.993042,26.826081,0.869781,26.826081,0.981113,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,PENN,34.65,N/A,16.600000,0.353877,1.480000,0.176938,0.955400,0.132207,6.875277,0.149105,3.694826,0.242545,N/A
499,VNO,30.60,N/A,49.310000,0.900596,1.200000,0.128231,3.560000,0.576541,15.036700,0.538767,7.911411,0.568588,N/A
500,ALK,45.81,N/A,12.720000,0.244533,1.600000,0.205765,0.798100,0.105368,9.247482,0.256461,0.737681,0.015905,N/A
501,PVH,63.44,N/A,4.540000,0.045726,0.788000,0.083499,0.460200,0.039761,4.758376,0.073559,1.103400,0.05169,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares To Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,165.39,N/A,27.230000,0.634195,39.430000,0.970179,7.110000,0.823062,20.645155,0.725646,15.693638,0.880716,0.806759
1,MSFT,283.58,N/A,29.220000,0.675944,12.750000,0.87674,10.740000,0.904573,21.015992,0.729622,15.052025,0.868787,0.811133
2,AMZN,135.75,N/A,3.280000,0.039761,10.750000,0.836978,2.880000,0.484095,25.330172,0.840954,6.740078,0.469185,0.534195
3,TSLA,900.61,N/A,108.720000,0.980119,26.100000,0.940358,14.660000,0.946322,70.097480,0.984095,54.744528,0.998012,0.969781
4,BRK.B,296.30,N/A,26.826081,0.615308,26.826081,0.949304,26.826081,0.993042,26.826081,0.869781,26.826081,0.981113,0.88171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,PENN,34.65,N/A,16.600000,0.353877,1.480000,0.176938,0.955400,0.132207,6.875277,0.149105,3.694826,0.242545,0.210934
499,VNO,30.60,N/A,49.310000,0.900596,1.200000,0.128231,3.560000,0.576541,15.036700,0.538767,7.911411,0.568588,0.542545
500,ALK,45.81,N/A,12.720000,0.244533,1.600000,0.205765,0.798100,0.105368,9.247482,0.256461,0.737681,0.015905,0.165606
501,PVH,63.44,N/A,4.540000,0.045726,0.788000,0.083499,0.460200,0.039761,4.758376,0.073559,1.103400,0.05169,0.058847


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace=True, drop=True)


In [21]:
rv_dataframe

,Ticker,Price,Number of Shares To Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,53.55,N/A,4.65,0.047714,0.7520,0.079523,0.7775,0.101392,2.789291,0.021869,0.762358,0.017893,0.053678
1,PVH,63.44,N/A,4.54,0.045726,0.7880,0.083499,0.4602,0.039761,4.758376,0.073559,1.103400,0.05169,0.058847
2,GM,38.59,N/A,7.07,0.101392,0.8354,0.087475,0.4162,0.031809,2.076869,0.015905,1.653190,0.087475,0.064811
3,LNC,51.80,N/A,7.61,0.109344,0.6000,0.073559,0.4635,0.04175,5.479752,0.095427,0.472241,0.003976,0.064811
4,PRU,104.43,N/A,8.00,0.115308,0.8863,0.091451,0.5600,0.059642,5.397487,0.093439,0.555955,0.007952,0.073559
5,SYF,34.77,N/A,4.99,0.055666,1.2900,0.138171,0.9376,0.127237,3.164824,0.027833,1.040742,0.043738,0.078529
6,PARA,24.85,N/A,4.07,0.04175,0.6921,0.077535,0.5569,0.057654,7.413304,0.182903,0.937034,0.035785,0.079125
7,MET,63.31,N/A,8.24,0.121272,0.9670,0.10338,0.7429,0.097416,4.701704,0.067594,0.726144,0.013917,0.080716
8,F,15.79,N/A,5.32,0.065606,1.3700,0.154076,0.3926,0.027833,4.029461,0.047714,2.106774,0.117296,0.082505
9,COF,112.37,N/A,4.43,0.043738,0.7800,0.081511,1.3300,0.205765,2.841608,0.023857,1.272425,0.059642,0.082903


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [22]:
for i in rv_dataframe.index:
    rv_dataframe.loc[i, 'Number of Shares To Buy'] = math.floor(position_size / rv_dataframe.loc[i, 'Price'])

In [23]:
rv_dataframe

,Ticker,Price,Number of Shares To Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,53.55,933,4.65,0.047714,0.7520,0.079523,0.7775,0.101392,2.789291,0.021869,0.762358,0.017893,0.053678
1,PVH,63.44,788,4.54,0.045726,0.7880,0.083499,0.4602,0.039761,4.758376,0.073559,1.103400,0.05169,0.058847
2,GM,38.59,1295,7.07,0.101392,0.8354,0.087475,0.4162,0.031809,2.076869,0.015905,1.653190,0.087475,0.064811
3,LNC,51.80,965,7.61,0.109344,0.6000,0.073559,0.4635,0.04175,5.479752,0.095427,0.472241,0.003976,0.064811
4,PRU,104.43,478,8.00,0.115308,0.8863,0.091451,0.5600,0.059642,5.397487,0.093439,0.555955,0.007952,0.073559
5,SYF,34.77,1438,4.99,0.055666,1.2900,0.138171,0.9376,0.127237,3.164824,0.027833,1.040742,0.043738,0.078529
6,PARA,24.85,2012,4.07,0.04175,0.6921,0.077535,0.5569,0.057654,7.413304,0.182903,0.937034,0.035785,0.079125
7,MET,63.31,789,8.24,0.121272,0.9670,0.10338,0.7429,0.097416,4.701704,0.067594,0.726144,0.013917,0.080716
8,F,15.79,3166,5.32,0.065606,1.3700,0.154076,0.3926,0.027833,4.029461,0.047714,2.106774,0.117296,0.082505
9,COF,112.37,444,4.43,0.043738,0.7800,0.081511,1.3300,0.205765,2.841608,0.023857,1.272425,0.059642,0.082903


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [24]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, 'Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_format = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {  
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['Price-to-Earnings Ratio', float_format],
    'E': ['PE Percentile', percent_format],
    'F': ['Price-to-Book Ratio', float_format],
    'G': ['PB Percentile', percent_format],
    'H': ['Price-to-Sales Ratio', float_format],
    'I': ['PS Percentile', percent_format],
    'J': ['EV/EBITDA', float_format],
    'K': ['EV/EBITDA Percentile', percent_format],
    'L': ['EV/GP', float_format],
    'M': ['EV/GP Percentile', percent_format],
    'N': ['RV Score', percent_format]
    }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [27]:
writer.save()

In [28]:
rv_dataframe

,Ticker,Price,Number of Shares To Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,53.55,933,4.65,0.047714,0.7520,0.079523,0.7775,0.101392,2.789291,0.021869,0.762358,0.017893,0.053678
1,PVH,63.44,788,4.54,0.045726,0.7880,0.083499,0.4602,0.039761,4.758376,0.073559,1.103400,0.05169,0.058847
2,GM,38.59,1295,7.07,0.101392,0.8354,0.087475,0.4162,0.031809,2.076869,0.015905,1.653190,0.087475,0.064811
3,LNC,51.80,965,7.61,0.109344,0.6000,0.073559,0.4635,0.04175,5.479752,0.095427,0.472241,0.003976,0.064811
4,PRU,104.43,478,8.00,0.115308,0.8863,0.091451,0.5600,0.059642,5.397487,0.093439,0.555955,0.007952,0.073559
5,SYF,34.77,1438,4.99,0.055666,1.2900,0.138171,0.9376,0.127237,3.164824,0.027833,1.040742,0.043738,0.078529
6,PARA,24.85,2012,4.07,0.04175,0.6921,0.077535,0.5569,0.057654,7.413304,0.182903,0.937034,0.035785,0.079125
7,MET,63.31,789,8.24,0.121272,0.9670,0.10338,0.7429,0.097416,4.701704,0.067594,0.726144,0.013917,0.080716
8,F,15.79,3166,5.32,0.065606,1.3700,0.154076,0.3926,0.027833,4.029461,0.047714,2.106774,0.117296,0.082505
9,COF,112.37,444,4.43,0.043738,0.7800,0.081511,1.3300,0.205765,2.841608,0.023857,1.272425,0.059642,0.082903
